# Lesson 3 Exercise 2: Focus on Primary Key
![](../images/cassandralogo.png)

Walk through the basics of creating a table with a good Primary Key in Apache Cassandra, inserting rows of data, and doing a simple CQL query to validate the information. 

Replace ##### with your own answers. 

In [ ]:
from src.database import (
    get_cs_session,
    insert_cs_rows,
    create_cs_keyspace,
    create_cs_table,
    drop_cs_table,
    drop_cs_keyspace,
    set_cs_keyspace,
    close_cs_session,
)

session = get_cs_session()
keyspace_name = "my_keyspace"
drop_cs_keyspace(session, keyspace_name)  # drop any existing keyspace (nice for reruns)
create_cs_keyspace(session, keyspace_name)
set_cs_keyspace(session, keyspace_name)  # set the keyspace to the one we just created
tables = set()

Imagine you need to create a new Music Library of albums 

Here is the information asked of the data:
1. Give every album in the music library that was created by a given artist
```sql
SELECT * FROM music_library WHERE artist_name=”The Beatles”
```


### Here is the collection of data
![](images/table3.png)

#### Practice by making the PRIMARY KEY only 1 Column (not 2 or more)

In [ ]:
table_name_alb = "album_library"
tables.add(table_name_alb)
drop_cs_table(session, table_name_alb)  # drop the table if it exists
column_names = [
    "year",
    "artist_name",
    "album_name",
    "city",
]
column_types = [
    "int",
    "text",
    "text",
    "text",
]
columns_alb = dict(zip(column_names, column_types))
primary_keys = ["artist_name"]
create_cs_table(
    session,
    table_name_alb,
    columns_alb,
    primary_keys,
)

### Let's insert the data into the table

In [ ]:
insert_cs_rows(
    session,
    table_name_alb,
    list(columns_alb.keys()),
    [
        (1970, "The Beatles", "Let It Be", "Liverpool"),
        (1965, "The Beatles", "Rubber Soul", "Oxford"),
        (1965, "The Who", "My Generation", "London"),
        (1966, "The Monkees", "The Monkees", "Los Angeles"),
        (1970, "The Carpenters", "Close To You", "San Diego"),
    ],
)

### Validate the Data Model -- Does it give you two rows?

In [ ]:
query = f"SELECT * FROM {table_name_alb} WHERE artist_name='The Beatles'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.year, row.artist_name, row.album_name, row.city)

If you used just one column as your PRIMARY KEY, your output should be:
1965 The Beatles Rubber Soul Oxford


That didn't work out as planned! Why is that?  Did you create a unique primary key?

## Try again 
Create a new table with a composite key this time

In [ ]:
# Drop and recreate keyspace to ensure clean schema
table_name_alb2 = "album_library2"
tables.add(table_name_alb2)
drop_cs_table(session, table_name_alb2)  # drop the table if it exists
column_names = [
    "year",
    "artist_name",
    "album_name",
    "city",
]
column_types = [
    "int",
    "text",
    "text",
    "text",
]
columns_alb2 = dict(zip(column_names, column_types))
primary_keys = ["artist_name", "year"]
create_cs_table(
    session,
    table_name_alb2,
    columns_alb2,
    primary_keys,
)

In [ ]:
insert_cs_rows(
    session,
    table_name_alb2,
    list(columns_alb2.keys()),
    [
        (1970, "The Beatles", "Let It Be", "Liverpool"),
        (1965, "The Beatles", "Rubber Soul", "Oxford"),
        (1965, "The Who", "My Generation", "London"),
        (1966, "The Monkees", "The Monkees", "Los Angeles"),
        (1970, "The Carpenters", "Close To You", "San Diego"),
    ],
)

### Validate the Data Model -- Did it work?

In [ ]:
query = f"SELECT * FROM {table_name_alb2} WHERE artist_name='The Beatles'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.year, row.artist_name, row.album_name, row.city)

### Your output should be:
1970 The Beatles Let it Be Liverpool<br>
1965 The Beatles Rubber Soul Oxford

### Drop the tables

In [ ]:
for table in tables:
    drop_cs_table(session, table)
close_cs_session(session)